In [22]:
# !pip install cartopy

In [23]:
import pandas as pd
import numpy as np

# plotting not yet supported
from cartopy import crs as ccrs, feature as cfeature 
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt

In [24]:
import pycountry    # finding country names
import difflib      # quanitfying string differences

In [25]:
# Function to create new columns for difference, percent change, and percentage makeup
def add_difference_and_percent(df, col1, col2):
    # Calculate the difference
    diff_col_name = f'{col1}_minus_{col2}'
    df[diff_col_name] = df[col1] - df[col2]
    
    # Calculate the percent change
    pct_change_col_name = f'{col1}_pct_change_{col2}'
    df[pct_change_col_name] = (df[col1] - df[col2]) / df[col2] * 100


    # Calculate the percent of previous
    # pct_prev_col_name = f'{col1}_pct_previous_{col2}'
    # df[pct_prev_col_name] = df[col1] / df[col2] * 100
    
    return df

In [26]:
volumes = pd.read_csv("./data/country_percent_scale_080124.csv")    # read in volumes of production in 2023
volumes

,iso3_country,actual_production_boed,expected_production_boed,percent_production_modeled
0,AGO,1.196436e+06,1.196436e+06,1.000000
1,ALB,1.121734e+04,1.121734e+04,1.000000
2,ARE,5.519692e+06,5.519692e+06,1.000000
3,ARG,1.355164e+06,1.355164e+06,1.000000
4,AUS,2.124336e+06,2.124336e+06,1.000000
...,...,...,...,...
80,USA,2.973635e+07,3.353577e+07,0.886705
81,UZB,8.005956e+05,8.005956e+05,1.000000
82,VEN,8.005084e+05,8.005084e+05,1.000000
83,VNM,2.840607e+05,2.840607e+05,1.000000


In [27]:
new_prod_in = pd.read_csv("./data/country-climate-trace_oil-and-gas-production_100824.csv")
new_trans_in = pd.read_csv("./data/country-climate-trace_oil-and-gas-transport_100824.csv")

# trace22_in = pd.read_csv("./data/trace22.csv") 
trace22_in = pd.read_csv("./data/oil-and-gas-production-and-transport_country_emissions.csv") 

In [28]:
new_prod_in.dropna(subset=['start_date'], inplace=True)
new_prod = new_prod_in[
    new_prod_in['start_date'].str.contains('2023')
]
new_prod = new_prod[
    new_prod['end_date'].str.contains('2023')
]

new_trans_in.dropna(subset=['start_date'], inplace=True)
new_trans = new_trans_in[
    new_trans_in['start_date'].str.contains('2023')
]
new_trans = new_trans[
    new_trans['end_date'].str.contains('2023')
]

In [29]:
# country aggregated former run


trace22 = trace22_in[['iso3_country','end_time','gas','emissions_quantity']]            # restrict to only relevant columns
trace22 = trace22[trace22['end_time'].str.contains('2022')].drop(columns=['end_time'])  # keep only items that are from 2022 run and drop time
trace22 = trace22[trace22['gas'].isin(['co2','ch4'])]                                   # restrict to only carbon dioxide and methane

# pivot table and reset index to mimic other format
trace22 = trace22.pivot(                                                                
    index='iso3_country', columns='gas', values='emissions_quantity'
).reset_index()

# give column names for the gases
trace22.columns.name = None                                                         
trace22 = trace22.rename(columns={
    'co2': 'CO2_trace22_total',
    'ch4': 'CH4_trace22_total'
})

trace22 = trace22.replace(0,np.nan)   

In [30]:
new_prod.set_index('iso3_country', inplace=True)
new_trans.set_index('iso3_country', inplace=True)
trace22.set_index('iso3_country', inplace=True)

# Join dataframes
joined = new_prod.join(
    new_trans, how='outer',
    lsuffix='_prod',rsuffix='_trans').join(
        trace22, how='outer')

joined.reset_index(inplace=True)

In [31]:
joined

,iso3_country,start_date_prod,end_date_prod,CO2_emissions_tonnes_prod,CH4_emissions_tonnes_prod,N2O_emissions_tonnes_prod,total_CO2e_100yrGWP_prod,total_CO2e_20yrGWP_prod,start_date_trans,end_date_trans,CO2_emissions_tonnes_trans,CH4_emissions_tonnes_trans,N2O_emissions_tonnes_trans,total_CO2e_100yrGWP_trans,total_CO2e_20yrGWP_trans,CH4_trace22_total,CO2_trace22_total
0,ABW,2023-01-01,2023-12-31,NaN,NaN,NaN,NaN,NaN,2023-01-01,2023-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,2023-01-01,2023-12-31,7.180143e+04,2685.643328,0.0,1.518336e+05,2.933670e+05,2023-01-01,2023-12-31,6.504703e+04,483.921676,0.0,7.946789e+04,1.049706e+05,NaN,NaN
2,AGO,2023-01-01,2023-12-31,1.343289e+07,568004.061806,0.0,3.035941e+07,6.029323e+07,2023-01-01,2023-12-31,8.028758e+06,1252.094929,0.0,8.066071e+06,8.132056e+06,514227.525006,1.551758e+07
3,AIA,2023-01-01,2023-12-31,NaN,NaN,NaN,NaN,NaN,2023-01-01,2023-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ALA,2023-01-01,2023-12-31,NaN,NaN,NaN,NaN,NaN,2023-01-01,2023-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,XKX,2023-01-01,2023-12-31,NaN,NaN,NaN,NaN,NaN,2023-01-01,2023-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,YEM,2023-01-01,2023-12-31,7.963005e+05,29784.631029,0.0,1.683883e+06,3.253533e+06,2023-01-01,2023-12-31,7.213920e+05,5366.843915,0.0,8.813240e+05,1.164157e+06,163956.998302,3.894952e+06
248,ZAF,2023-01-01,2023-12-31,NaN,NaN,NaN,NaN,NaN,2023-01-01,2023-12-31,NaN,NaN,NaN,NaN,NaN,2878.461559,1.172470e+05
249,ZMB,2023-01-01,2023-12-31,NaN,NaN,NaN,NaN,NaN,2023-01-01,2023-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
joined['CO2_trace_new_total'] = joined['CO2_emissions_tonnes_prod'] + joined['CO2_emissions_tonnes_trans']
joined['CH4_trace_new_total'] = joined['CH4_emissions_tonnes_prod'] + joined['CH4_emissions_tonnes_trans']

merged = joined[
    ['iso3_country',
     'CO2_emissions_tonnes_prod', 'CO2_emissions_tonnes_trans',
     'CH4_emissions_tonnes_prod', 'CH4_emissions_tonnes_trans',
     'CO2_trace_new_total', 'CH4_trace_new_total',
     'CO2_trace22_total', 'CH4_trace22_total']
]

world_row = pd.DataFrame([merged.sum()],columns=merged.columns)
world_row['iso3_country'] = 'World'
merged = pd.concat([merged,world_row], ignore_index=True)

In [33]:
merged

,iso3_country,CO2_emissions_tonnes_prod,CO2_emissions_tonnes_trans,CH4_emissions_tonnes_prod,CH4_emissions_tonnes_trans,CO2_trace_new_total,CH4_trace_new_total,CO2_trace22_total,CH4_trace22_total
0,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,7.180143e+04,6.504703e+04,2.685643e+03,4.839217e+02,1.368485e+05,3.169565e+03,NaN,NaN
2,AGO,1.343289e+07,8.028758e+06,5.680041e+05,1.252095e+03,2.146165e+07,5.692562e+05,1.551758e+07,5.142275e+05
3,AIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ALA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
247,YEM,7.963005e+05,7.213920e+05,2.978463e+04,5.366844e+03,1.517693e+06,3.515147e+04,3.894952e+06,1.639570e+05
248,ZAF,NaN,NaN,NaN,NaN,NaN,NaN,1.172470e+05,2.878462e+03
249,ZMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250,ZWE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
merged = add_difference_and_percent(merged,'CO2_trace_new_total','CO2_trace22_total')       # adding columns for production diff in carbon dioxide
merged = add_difference_and_percent(merged,'CH4_trace_new_total','CH4_trace22_total')       # adding columns for production diff in methane


In [35]:
merged.to_csv('./data/new_method_&_v3_100924.csv',index=False) # write csv of this